In [1]:
import pandas as pd
import numpy as np

<h1>Rotten movies</h1>

In [2]:
from datasets import load_dataset

rotten_data = load_dataset("rotten_tomatoes")

Downloading:   0%|          | 0.00/1.89k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/921 [00:00<?, ?B/s]

Using custom data configuration default


Downloading:   0%|          | 0.00/488k [00:00<?, ?B/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

0 examples [00:00, ? examples/s]

Dataset rotten_tomatoes_movie_review downloaded and prepared to /home/diego/.cache/huggingface/datasets/rotten_tomatoes_movie_review/default/1.0.0/40d411e45a6ce3484deed7cc15b82a53dad9a72aafd9f86f8f227134bec5ca46. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [ ]:
print(f"Total length of polarity rotten test {len(rotten_data['test'])}")
print(f"Total length of polarity rotten train {len(rotten_data['train'])}")

we keep train for training and test for testing as it is

In [3]:
# we merge train and test

texts = rotten_data["test"]["text"]
labels = rotten_data["test"]["label"]

d = {"text": texts,
     "label": labels,
}

df_rotten = pd.DataFrame(data=d)
df_rotten["sentiment"] = df_rotten.apply(lambda row: "positive" if row['label'] else "negative", axis=1)
df_rotten.head(2)

Total length of polarity yelp test 1066



,text,label,sentiment
0,lovingly photographed in the manner of a golde...,1,positive
1,consistently clever and suspenseful .,1,positive


In [5]:
len(rotten_data['train'])

8530

In [ ]:
texts = df_yelp["text"].values
lens = [len(el.split(" ")) for el in texts]
print(f"Max len: {np.max(lens)}")
print(f"Mean len: {np.mean(lens)}")
print(f"Min len: {np.min(lens)}")

df_yelp["sentiment"].value_counts()

In [ ]:
# filter out reviews longer than 2000 and shorter than 10 words
texts = df_yelp["text"].values
df_yelp["review_len"] = [len(el.split(" ")) for el in texts]
df_yelp = df_yelp[(df_yelp["review_len"] <= 2000) & (df_yelp["review_len"] >= 10)].copy()

lens = [len(el) for el in df_yelp["text"].values]
print(f"Max len: {np.max(lens)}")
print(f"Mean len: {np.mean(lens)}")
print(f"Min len: {np.min(lens)}")

df_yelp["label"].value_counts()

In [ ]:
df_yelp["label_counter"] = [int(not el) for el in df_yelp["label"]]
df_yelp["sentiment_counter"] = ["positive" if x=="negative" else "negative" for x in df_yelp["sentiment"]]
df_yelp.rename(columns={"label": "label_ex",
                        "text": "example",
                        "sentiment": "sentiment_ex"}, inplace=True)
df_yelp.head(2)

In [ ]:
df_pos = df_yelp[df_yelp["label_ex"] == 1].copy()
df_neg = df_yelp[df_yelp["label_ex"] == 0].copy()

## Sample 10k instances - 5k positive, 5k negative. Train-test split 80%-20%

In [ ]:
seed = 2023
# sample positive and negative labels
data_pos = df_pos.sample(n=5000, replace=False, random_state=seed)
data_neg = df_neg.sample(n=5000, replace=False, random_state=seed)

# sample train-test split positive labels (80-20)%
test_pos = data_pos.sample(frac=0.2, replace=False, random_state=seed)
train_pos = data_pos[~data_pos.index.isin(test_pos.index)]

# sample train-test split negative labels (80-20)%
test_neg = data_neg.sample(frac=0.2, replace=False, random_state=seed)
train_neg = data_neg[~data_neg.index.isin(test_neg.index)]

# build train and test
testset = test_neg.append(test_pos)
trainset = train_neg.append(train_pos)

print()
print(f"len test: {len(testset)}")
print(f"len train: {len(trainset)}")

In [ ]:
testset.to_csv("../yelp/test.csv", sep="\t", index=False)
trainset.to_csv("../yelp/train.csv", sep="\t", index=False)

In [ ]:
def produce_datasets(n, m, out_dir):
     train_set = pd.read_csv("../yelp/train.csv", sep="\t")
     print(len(train_set))

     df_origin = train_set.sample(n=n+m, replace=False, random_state=seed)
     df_n = df_origin.sample(n=n, replace=False, random_state=seed)
     df_seed = df_n.sample(n=m, replace=False, random_state=seed)

     print(f"len origin: {len(df_origin)}")
     print(f"len n_data: {len(df_n)}")
     print(f"len seed: {len(df_seed)}")

     df_origin = trainset.sample(n=n+m, replace=False, random_state=seed)
     df_n = df_origin.sample(n=n, replace=False, random_state=seed)
     df_seed = df_n.sample(n=m, replace=False, random_state=seed)

     print(f"len origin: {len(df_origin)}")
     print(f"len n_data: {len(df_n)}")
     print(f"len seed: {len(df_seed)}")

     df_seed["paired_id"] = [i for i in range(len(df_seed))]
     df_seed["counterfactual"] = ["None" for i in range(len(df_seed))]

     df_origin.to_csv(f"../yelp/{out_dir}/origin_data.csv", sep="\t", index=False)
     df_n.to_csv(f"../yelp/{out_dir}/n_data.csv", sep="\t", index=False)
     df_seed.to_csv(f"../yelp/{out_dir}/seed_data.csv", sep="\t", index=False)

## n=5k & m=2.5k (n=2m)

In [ ]:
produce_datasets(5000, 2500, "tr2")

In [ ]:
produce_datasets(2000, 1000, "tr1")

The final size of each training set will be m+m+m=2m+m=n+m. The following sets are sampled from train.csv

- origin.csv stores n+m original data points
- m_data.csv
- seed_data.csv is a sample (size m) from n_data

You now need to produce m countefactuals from seed_data!! You will then use n_data.csv and m generated counterfactuals to train your classfier